In [0]:
from sklearn import preprocessing
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import pandas as pd
from tqdm import tqdm
from gensim.models import Word2Vec 
import gensim
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
df_train=pd.read_csv("project_data_train_dssm2.csv")



In [0]:
Label = preprocessing.LabelEncoder()
search_column=Label.fit_transform(df_train['search_term'])
df_train['label_search']=search_column



In [0]:
df_train['product_uid']= df_train['product_uid'].astype(str)
df_train['label_search']= df_train['label_search'].astype(str)

In [0]:
label_search_unique = df_train['label_search'].unique()

In [0]:
train_data = []

# populate the list with the product codes
for i in tqdm(label_search_unique):
    temp = df_train[df_train["label_search"] == i]["product_uid"].tolist()
    train_data.append(temp)


100%|██████████| 11108/11108 [00:44<00:00, 247.66it/s]

In [0]:
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14,min_count=1)

model.build_vocab(train_data, progress_per=200)

model.train(train_data, total_examples = model.corpus_count, epochs=10, report_delay=1)


(564390, 564390)

In [0]:
model.wv.vocab.keys()

dict_keys(['100001', '100664', '100672', '100739', '101036', '101370', '101440', '101534', '102239', '103250', '103262', '100531', '100533', '100700', '101075', '102864', '102914', '100002', '100092', '101629', '101795', '103015', '103057', '103356', '103384', '100005', '100331', '100616', '101126', '101287', '102542', '102654', '102941', '104358', '105204', '122491', '124091', '140835', '100006', '101635', '105261', '159017', '101209', '101272', '101442', '107408', '112968', '133247', '100871', '100924', '100944', '101265', '101612', '102909', '103000', '100007', '100058', '101502', '101836', '101894', '102044', '102098', '102418', '102599', '100009', '100865', '107285', '111072', '111098', '132786', '133462', '100010', '100790', '102221', '153601', '175540', '179690', '184373', '100011', '105641', '100804', '101019', '101911', '105795', '137112', '100592', '100809', '101063', '101415', '102290', '100012', '202245', '100013', '107177', '111749', '119721', '123157', '142327', '149903',

In [0]:
model.save("mymodel_new.pkl")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
loaded_model = pickle.load(open('mymodel_new.pkl', 'rb'))

In [0]:
from google.colab import files
files.download('mymodel_new.pkl') 

In [0]:
model_keys=model.wv.vocab

In [0]:
list1_temp=[]
list2_temp=[]
for x in model_keys:
  df_temp=df_train[df_train['product_uid']==str(x)]
  list1_temp.append([value for value in df_temp['search_term'].values])
  list2_temp.append(x)

id_=list2_temp
value=list1_temp
final_dict=dict(zip(id_,value))

In [0]:
df_train2=df_train[['product_uid','product_title']].drop_duplicates()
id_=df_train2['product_uid']
value=df_train2['product_title']
final_dict2=dict(zip(id_,value))

In [0]:
def get_key2(val):
  dict_key=''
  temp_list=[]
  for key, value in final_dict.items():
    for x in value:
      if val in value:
        dict_key=str(key)
  for x in model.wv.most_similar(dict_key):
    temp_list.append(final_dict2[x[0]])

  return(temp_list[:5])
 

In [0]:
df_train

,product_uid,product_title,search_term,relevance,brand,product_description,label_search
0,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,Simpson Strong-Tie,"Not only do angles make joints stronger, they ...",2076
1,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,Simpson Strong-Tie,"Not only do angles make joints stronger, they ...",6111
2,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DeckOver,BEHR Premium Textured DECKOVER is an innovativ...,3685
3,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Delta,Update your bathroom with the Delta Vero Singl...,8177
4,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Delta,Update your bathroom with the Delta Vero Singl...,9002
...,...,...,...,...,...,...,...
56434,206638,Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...,tv riser glass,1.00,Atlantic,"Atlantic, Inc. 94835722 Uniquely designed for ...",10159
56435,206639,Philips 40-Watt Halogen R20 Flood Light Bulb (...,r20 halogen light,3.00,Philips,Philips Energy Advantage lamps use less energy...,8147
56436,206641,Schlage Camelot In-Active Aged Bronze Handlese...,schlage lock siena half dummy knob with,2.33,Schlage,The Schlage Camelot In-Active Aged Bronze Hand...,8765
56437,206648,Plastec 11 in. x 24 in. Rose Garden Wall Decor...,zen garden decor,3.00,Plastec,The Rose Garden is inspired by the popular ear...,11092
